#### **Ingesta del archivo "movie_genre.json"**

In [0]:
dbutils.widgets.text("environment","")
var_environment = dbutils.widgets.get("environment")

In [0]:
dbutils.widgets.text("file_date","2024-12-30")
var_file_date = dbutils.widgets.get("file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

##### Librerías

In [0]:
from pyspark.sql.functions import lit,current_timestamp
from pyspark.sql.types import StructType,StructField,IntegerType

##### Esquema

In [0]:
movie_genre_schema = StructType([
    StructField("movieId",IntegerType(),False),
    StructField("genreId",IntegerType(),True)
])

##### Leer fichero JSON

In [0]:
df_movie_genre = spark.read.schema(movie_genre_schema).json(f"{bronze_folder_path}/{var_file_date}/movie_genre.json")

##### Renombrar y añadir columnas

In [0]:
df_movie_genre_final = add_ingestion_date(df_movie_genre).withColumnsRenamed({"movieId":"movie_id","genreId":"genre_id"}) \
                                     .withColumn("environment",lit(var_environment)).withColumn("file_date",lit(var_file_date))

##### Escribir fichero en Silver (particionado)

In [0]:
merge_condition = "target.movie_id = source.movie_id AND target.genre_id = source.genre_id AND target.file_date = source.file_date"
merge_delta_lake(df_movie_genre_final,"movie_silver","movie_genres",silver_folder_path,merge_condition,"file_date")

In [0]:
%sql
SELECT COUNT(1) AS TOTAL, file_date
FROM movie_silver.movie_genres
GROUP BY file_date

In [0]:
#df_movie_genre_final.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.movie_genres")

In [0]:
dbutils.notebook.exit("Success")